### Load Input Data

In [1]:
PROCESSED_DATA = "./processed-data"

import torch
import pandas as pd
import numpy as np

dataset = pd.read_csv('{}/latestSequence.csv'.format(PROCESSED_DATA), header = 0)
dataset.set_index(dataset.columns[0], inplace=True)
print(dataset[-24:])
input_data = np.array(dataset)
mean = np.mean(input_data, axis=0)
std = np.std(input_data, axis=0) 
input_data = (input_data - mean)/std  
input_data = torch.Tensor(input_data).unsqueeze(0)
print(input_data.shape)

                  Temp (°C)  Dew Point Temp (°C)  Rel Hum (%)  \
Date/Time                                                       
2019-08-21 00:00       23.1                 21.7         92.0   
2019-08-21 01:00       23.3                 21.8         91.0   
2019-08-21 02:00       23.2                 22.1         94.0   
2019-08-21 03:00       22.9                 22.7         99.0   
2019-08-21 04:00       22.6                 22.6        100.0   
2019-08-21 05:00       22.6                 22.6        100.0   
2019-08-21 06:00       22.8                 21.7         94.0   
2019-08-21 07:00       20.6                 20.6        100.0   
2019-08-21 08:00       20.4                 20.4        100.0   
2019-08-21 09:00       21.7                 21.7        100.0   
2019-08-21 10:00       23.0                 21.9         93.0   
2019-08-21 11:00       23.8                 20.9         84.0   
2019-08-21 12:00       23.9                 22.8         94.0   
2019-08-21 13:00       25

### Set Model

In [4]:
import torch
import torch.nn as nn

class SimpleLSTM(nn.Module):
    def __init__(self, input_size = 7, output_size = 1, hidden_size=100, num_layers=1):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True) 
        self.fc = nn.Linear(self.hidden_size, self.output_size) 

    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, cell
    
    def forward(self, x):
        # hidden, cell state init
        h, c = self.init_hidden(x.size(0))
        h, c = h.to(x.device), c.to(x.device)
        out, (h, c) = self.lstm(x, (h, c))     
        final_output = self.fc(out[:, -1:, :])     
        final_output = torch.squeeze(final_output, dim = 1) # shape (100,1)

        return final_output

### Load Model and Predict  

The input data are 24 set of 480 hours data till 00, 01, ...,  23 hour on Aug. 21th (Yesterday)  
The predictions are temperatures at 00 hour, 01 hour, ..., 23 hour on Aug. 22th (Today)

In [6]:
model_path = './saved/LSTM/best_model.pt'
model = SimpleLSTM()

checkpoint = torch.load(model_path)    
state_dict = checkpoint['net']   
model.load_state_dict(state_dict) 


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
preds = []

for count in range(0,24):
    pred = model(input_data[:, count:480+count, :])
    pred = pred.item() * std[0] + mean[0] # de-normalization
    preds.append(pred)

count = 0
for pred in preds:
    print("{:02d} - {:2.2f}".format(count, pred))
    count = count+1

# The actual temp was 17.0
# http://climate.weather.gc.ca/climate_data/hourly_data_e.html?hlyRange=2008-07-15%7C2019-08-07&dlyRange=2008-07-15%7C2019-08-07&mlyRange=%7C&StationID=47267&Prov=ON&urlExtension=_e.html&searchType=stnProv&optLimit=specDate&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=74&Month=8&Day=1&lstProvince=ON&timeframe=1&Year=2019


00 - 21.76
01 - 21.93
02 - 22.16
03 - 22.43
04 - 22.71
05 - 22.95
06 - 23.16
07 - 23.16
08 - 23.23
09 - 23.30
10 - 23.42
11 - 23.53
12 - 23.54
13 - 23.63
14 - 23.73
15 - 23.81
16 - 23.71
17 - 23.61
18 - 23.45
19 - 23.27
20 - 23.10
21 - 22.96
22 - 22.83
23 - 22.73


### Summary

1) LSTM Model was trained on Kingston Climate Station's data between 2015.1.1. ~ 2019.8.21  
   Weakness - The prediction is dependent on only one station in Kingston  
   
2) Model gets 480 hours data as input and predicts the temperature 24 hours later  